# Methoden-Übersicht
für das Fachpraktikum Simulation Multiagentensysteme
mit dem Thema Entwicklung eines Multiagentensystems zur kooperativen Erkundung in einer 2D-Simulationsumgebung

## Umgebungswahrnehmung

Umgebung: 2D-Moore-Grid

### Raycasting mit Bresenham-Algorithmus

Simuliert die Sichtlinie und Sensorreichweite im Grid, um festzustellen, welche Felder sichtbar sind. Der Bresenham-Algorithmus wird verwendet, um die Zellen zu bestimmen, die eine Linie zwischen zwei Punkten kreuzt, was für Grid-basierte Umgebungen effizient ist.

Vorteile:
* Effizient für Grid-basierte Umgebungen
* Einfach zu implementieren

Nachteile:
* Keine Berücksichtigung von komplexen Sichtmodellen / Sensorrauschen / ... möglich

## Zielfindung

### Frontier Detection

## Routenplanung und Kollisionsvermeidung